# 21. Class Metaprogramming

**클래스 메타프로그래밍**이란, 실행 도중 클래스를 생성하거나 커스터마이즈하는 기술을 말한다. 클래스는 파이썬의 일급 객체이므로 $\texttt{class}$라는 키워드를 사용하지 않고도 언제든 함수를 사용해서 생성할 수 있다.

메타클래스는 강력하지만, 제대로 사용하기는 어렵다. 클래스 데커레이터는 이와 같은 문제의 상당 부분을 해결한다. 사실 메타클래스는 실제 코딩에서는 정당화하기 굉장히 어려우며, 파이썬 2.6에 클래스 데커레이터가 등장함으로써 거의 쓰이지 않는 듯하다.

책에서는 다음과 같은 충고를 하고 있다: "프레임워크를 만들고 있지 않다면, (실제 업무 등에서) 메타클래스를 작성해서는 **안 된다.**"

## 클래스 팩토리

클래스 팩토리는 일종의 **함수**로서, 클래스명과 속성명을 전달하면 이름으로 항목을 가져올 수 있게 해 주고 디버깅하기 좋은 \_\_repr\_\_() 메서드를 제공하는 $\texttt{tuple}$의 서브클래스를 생성한다.

In [6]:
def record_factory(cls_name, field_names) :
    try :
        field_names = field_names.replace(',', ' ').split()
    except AttributeError :
        pass
    field_names = tuple(field_names)
    
    def __init__(self, *args, **kwargs) :
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items() :
            setattr(self, name, value)

    def __iter__(self) :
        for name in self.__slots__ :
            yield getattr(self, name)

    def __repr__(self) :
        values = ', '.join('{}={!r}'.format(*i)
                           for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)

    cls_attrs = dict(__slots__ = field_names, 
                     __init__ = __init__, 
                     __iter__ = __iter__, 
                     __repr__ = __repr__)

    return type(cls_name, (object,), cls_attrs)

In [12]:
Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
print(rex)

name, weight, _ = rex
print((name, weight))

print("{2}'s dog weights {1}kg".format(*rex))

Dog(name='Rex', weight=30, owner='Bob')
('Rex', 30)
Bob's dog weights 30kg


## 임포트 타임과 런타임

성공적으로 메타프로그래밍을 하려면, 파이썬 인터프리터가 언제 각 코드 블록을 평가하는지 알고 있어야 한다. 파이썬 프로그래머들은 '임포트 타임'과 '런타임'을 구분하지만, 이 용어들은 엄격히 정의되어 있지 않으며 구분이 모호한 경우도 있다. 임포트 타임에 인터프리터는 .py 모듈에 들어 있는 소스 코드를 위에서부터 한 번 파싱하고, 실행할 바이트코드를 생성한다. 구문 에러는 이때 발생한다.

컴파일 작업은 확실히 임포트 타임의 활동이긴 하지만, 이때 다른 일도 일어난다. import 문은 단지 단순한 선언이 아니며, 처음 임포트되는 모듈의 모든 최상위 수준 코드를 실제로 실행한다. 즉 import 문이 각종 '런타임'의 동작을 유발하기 때문에 '임포트 타임'과 '런타임'의 구분이 모호해지는 것이다.

## 메타클래스

메타클래스는 일종의 클래스 팩토리다. 다만 위 클래스 팩토리의 예시와 같은 함수 대신 클래스로 만들어진다는 점이 다르다. 파이썬 객체 모델을 생각해 보자. 클래스도 객체이므로, 각 클래스는 다른 어떤 클래스의 객체여야 한다. 기본적으로 파이썬 클래스는 type의 객체다. 즉, type는 대부분의 내장된 클래스와 사용자 정의 클래스에 대한 메타클래스다.

In [13]:
'spam'.__class__

str

In [14]:
str.__class__

type

In [15]:
int.__class__

type

In [16]:
type.__class__

type

무한 회귀를 방지하기 위해서, type은 자기 자신의 객체로 정의되어 있다.

정리하자면, 모든 클래스는 $\texttt{type}$의 객체지만, 메타클래스는 $\texttt{type}$의 서브클래스이기도 하므로 클래스 팩토리로서 행동한다. 특히 메타클래스는 \_\_init\_\_() 메서드를 구현함으로서 자신의 객체를 커스터마이즈할 수 있다.